In [1]:
# Imports for gplearn and pydotplus in order to see graph view

In [2]:

from IPython.display import Image
import pydotplus

from gplearn.genetic import SymbolicRegressor
from gplearn.fitness import make_fitness

In [3]:
#--Import the required libraries--
import math
import random
import matplotlib.pyplot as plt
import numpy as np

#--debug mode to report on evaluation of tree--
debug_eval = False



# Import Scipy generic dataset 
* Number of Instances:506
* Number of Attributes:13
* Attribute Information (in order):


<li>CRIM     per capita crime rate by town</li>
<li>ZN       proportion of residential land zoned for lots over 25,000 sq.ft.</li>
<li>INDUS    proportion of non-retail business acres per town</li>
<li>CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)</li>
<li>NOX      nitric oxides concentration (parts per 10 million)</li>
<li>RM       average number of rooms per dwelling</li>
<li>AGE      proportion of owner-occupied units built prior to 1940</li>
<li>DIS      weighted distances to five Boston employment centres</li>
<li>RAD      index of accessibility to radial highways</li>
<li>TAX      full-value property-tax rate per \\$10,000</li>
<li>PTRATIO  pupil-teacher ratio by town</li>
<li>B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town</li>
<li>LSTAT    \\% lower status of the population</li>
<li>MEDV     Median value of owner-occupied homes in $1000’s</li>

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston

#load the data from the default data set, and split it into a tuple
data = load_boston(return_X_y = True)

#what percent of our data do we want to use to validate
split_percent = 0.2
train_x, test_x, train_y, test_y = train_test_split(*data, test_size = split_percent, random_state = 0)

#print out the shapes for clarity
print("Shapes:\n data_x:{}\n data_y:{}\n train_x:{}\n test_x:{}\n train_y:{}\n test_y:{}"
      .format(data[0].shape,data[1].shape,train_x.shape,test_x.shape,train_y.shape,test_y.shape))

Shapes:
 data_x:(506, 13)
 data_y:(506,)
 train_x:(404, 13)
 test_x:(102, 13)
 train_y:(404,)
 test_y:(102,)


# Symbolic regression

We run a symbolic regression on a 1000 individuals for 20 generations. There are three kinds of mutation, subtree, hoist, and point. The hoist mutation is the only type that differs significantly from what was talked about in class. In it, a random subtree is selected, then, a random subtree from within it is lifted to it’s root node. It is used to combat tree bloat, and it appears to be very effective. Without the hoist mutation, tree trees grew like redwoods.

In [ ]:
#This part sets up the symbolic regressor
est_gp = SymbolicRegressor(population_size=10000,
                           generations=40, stopping_criteria=0,
                           p_crossover=0.9, p_subtree_mutation=0.01,
                           p_hoist_mutation=0.01, p_point_mutation=0.01,
                           max_samples=0.6, verbose=1,
                           parsimony_coefficient=0.001, random_state=0,
                           metric = 'rmse',
                           function_set=('add', 'sub', 'mul', 'div',"sin"),
                           n_jobs = 10
                           )
#This part runs it on our data
est_gp.fit(train_x, train_y)

    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    18.39 3931369986002.4434        5 7.405075814309804 8.075799745618673     51.99s
   1     6.46 4277423.525192768       24 6.078022707279818 6.105147554124325      9.13m
   2      5.4 143.87782647919852       24 5.685162977936526 6.855180318004764     13.01m
   3     6.81 1386.0260507911844       26 5.241415543637552 7.2254326636846224     14.88m
   4     7.76 280.0639664766024       11 5.270680740203347 6.180452013360248     16.12m
   5    11.74 180.72981832106476       24 5.160452603983904 5.966489709315549     17.02m
   6    18.56 273.5285434394705       11 4.78794942210678 6.741768154362778     17.88m
   7    27.72 260.94217963947665       49 4.743968034216383 5.414616563096804     18.86m
   8     32.7 259.81546085673307       24 4.32

# Scoring
Returns the coefficient of determination R^2 of the prediction.

The coefficient R^2 is defined as (1 - u/v), where u is the residual sum of squares ((y_true - y_pred) ** 2).sum() and v is the total sum of squares ((y_true - y_true.mean()) ** 2).sum(). The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of y, disregarding the input features, would get a R^2 score of 0.0.

In [ ]:
 print(best_gp._program)
best_gp.score(test_x,test_y)

In [ ]:
graph = pydotplus.graphviz.graph_from_dot_data(best_gp._program.export_graphviz())
Image(graph.create_png())